# Семинар 1. Выкачиваем Интернет.

## Введение
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть специальный модуль **urllib.request**. 

## Минимальный пример
Допустим, мы хотим скачать главную страницу Хабрахабра.  
На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер запрос ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать запрос на сервер так же, как это делает браузер:

In [12]:
import urllib.request  # импортируем модуль 
req = urllib.request.Request('https://habrahabr.ru/')
with urllib.request.urlopen(req) as response:
   html = response.read().decode('utf-8')

В переменной **req** у нас как раз находится запрос.
Функция **urlopen** получает ответ сервера и скачивает страницу по ссылке https://habrahabr.ru/ в переменную **response**. **response** ведет себя как файл: например мы можем прочитать его содержимое с помощью **.read()** в другую переменную. 
Вот так просто мы сохранили код страницы в переменной **html**. Убедимся, что в там лежит html-код:

In [13]:
print(html[:210])

<!DOCTYPE html>
<html >
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
    <meta content='width=1024' name='viewport'>
    <title>Интересные публикации / Хабрахабр</title>





Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу с помощью **urllib**, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5". 
Но можно, например, представиться Мозиллой:

In [ ]:
url = 'https://habrahabr.ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

req = urllib.request.Request('https://habrahabr.ru/', headers={'User-Agent':user_agent})  
# добавили в запрос информацию о том, что мы браузер Мозилла

with urllib.request.urlopen(req) as response:
   html = response.read().decode('utf-8')

## Напоминание: как найти на странице что-нибудь
Теперь предположим, что мы хотим выкачивать заголовки статей с главной страницы Хабрахабра. Код страницы у нас уже есть, но как из него что-то вытащить. Для начала нужно посмотреть в [исходник](view-source:https://habrahabr.ru/) и заметить, что заголовки хранятся в тэге **h2** с классом **post__title**. Заголовок выглядит примерно так:

<h2 class="post__title">
      <a href="https://habrahabr.ru/flows/admin/" class="post__flow"  onclick="if (typeof ga === 'function') { ga('send', 'event', 'flow', 'feed page', 'Администрирование'); }">Администрирование</a><span class="post__title-arrow">&nbsp;&rarr;</span>
      <a href="https://habrahabr.ru/company/okmeter/blog/309600/" class="post__title_link">Мониторинг сетевого стэка linux</a>
    </h2>
 
А код у него такой:

In [ ]:
<h2 class="post__title">
      <a href="https://habrahabr.ru/flows/admin/" class="post__flow"  onclick="if (typeof ga === 'function') { ga('send', 'event', 'flow', 'feed page', 'Администрирование'); }">Администрирование</a><span class="post__title-arrow">&nbsp;&rarr;</span>
      <a href="https://habrahabr.ru/company/okmeter/blog/309600/" class="post__title_link">Мониторинг сетевого стэка linux</a>
    </h2>

Чтобы вытащить все такие заголовки, воспользуемся регулярным выражением.

In [19]:
import re
regPostTitle = re.compile('<h2 class="post__title">.*?</h2>', flags=re.U | re.DOTALL)
titles = regPostTitle.findall(html)

Посмотрим, сколько там заголовков. И взглянем, например, на первые три.

In [20]:
print(len(titles))

10


In [21]:
print(titles[:3])

['<h2 class="post__title">\n      <a href="https://habrahabr.ru/flows/develop/" class="post__flow"  onclick="if (typeof ga === \'function\') { ga(\'send\', \'event\', \'flow\', \'feed page\', \'Разработка\'); }">Разработка</a><span class="post__title-arrow">&nbsp;&rarr;</span>\n      <a href="https://habrahabr.ru/company/yandex/blog/309652/" class="post__title_link">Выявление проблем дорожной сети с помощью Яндекс.Пробок. Лекция в Яндексе</a>\n      \n\n\n\n\n\n\n\n\n    </h2>', '<h2 class="post__title">\n      <a href="https://habrahabr.ru/flows/develop/" class="post__flow"  onclick="if (typeof ga === \'function\') { ga(\'send\', \'event\', \'flow\', \'feed page\', \'Разработка\'); }">Разработка</a><span class="post__title-arrow">&nbsp;&rarr;</span>\n      <a href="https://habrahabr.ru/post/309660/" class="post__title_link">Plug-and-Get-Security I, мониторинг настроек TLS в роще доменов</a>\n      \n\n\n\n<a href="/sandbox/" class="flag flag_sandbox" title="Перейти в песочницу">из пес

Теперь давайте очистим заголовки от лишних переносов строк, лишних тэгов и распечатаем их подряд.

In [28]:
new_titles = []
regTag = re.compile('<.*?>', flags=re.U | re.DOTALL)
regSpace = re.compile('\s{2,}', flags=re.U | re.DOTALL)
for t in titles:
    clean_t = regSpace.sub("", t)
    clean_t = regTag.sub("", clean_t)
    new_titles.append(clean_t)
for t in new_titles:
    print(t)

Разработка&nbsp;&rarr;Выявление проблем дорожной сети с помощью Яндекс.Пробок. Лекция в Яндексе
Разработка&nbsp;&rarr;Plug-and-Get-Security I, мониторинг настроек TLS в роще доменовиз песочницы
Администрирование&nbsp;&rarr;Мониторинг сетевого стэка linux
Управление&nbsp;&rarr;Квалификация коллег-программистов: ожидание и реальностьиз песочницы
Разработка&nbsp;&rarr;Немного о ARM Security Extensions (aka ARM TrustZone)
Разработка&nbsp;&rarr;Offline-first приложение с Hoodie & React. Часть вторая: авторизацияtutorial
Администрирование&nbsp;&rarr;Кабель-менеджмент и PUE: как они связаны?
Разработка&nbsp;&rarr;Функциональная безопасность, Часть 2 из 2. МЭК 61508: кем быть, Шерлоком Холмсом или Дата Туташхиа?
Управление&nbsp;&rarr;Дорогие стартапы, хватит задавать математические задачки, чтобы понять умею-ли я программироватьперевод
Управление&nbsp;&rarr;Мнения экспертов об ушедшем в историю «правиле 20%» Google


Ну и осталось убрать некрасивые кусочки html, а именно заменить специальные html-последовательности nbsp и rarr на стрелочку, например.

In [29]:
for t in new_titles:
    print(t.replace("&nbsp;&rarr;", " -> "))

Разработка -> Выявление проблем дорожной сети с помощью Яндекс.Пробок. Лекция в Яндексе
Разработка -> Plug-and-Get-Security I, мониторинг настроек TLS в роще доменовиз песочницы
Администрирование -> Мониторинг сетевого стэка linux
Управление -> Квалификация коллег-программистов: ожидание и реальностьиз песочницы
Разработка -> Немного о ARM Security Extensions (aka ARM TrustZone)
Разработка -> Offline-first приложение с Hoodie & React. Часть вторая: авторизацияtutorial
Администрирование -> Кабель-менеджмент и PUE: как они связаны?
Разработка -> Функциональная безопасность, Часть 2 из 2. МЭК 61508: кем быть, Шерлоком Холмсом или Дата Туташхиа?
Управление -> Дорогие стартапы, хватит задавать математические задачки, чтобы понять умею-ли я программироватьперевод
Управление -> Мнения экспертов об ушедшем в историю «правиле 20%» Google


### Ура! Мы умеем скачивать страницу и вытаскивать из нее какую-то интересную информацию.

## Задания

1. Скачать главную страницу Яндекс.Погоды и <br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;а) распечатать сегодняшнюю температуру и облачность<br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;б) распечатать время восхода и заката<br>
    
    &nbsp;&nbsp;&nbsp;&nbsp;в) погоду на завтра<br>
    
2. Скачать главную страницу waitbutwhy.com. Распечатать заголовки популярных постов (которые в колонке справа с надписью Popular Posts) и колличество комментариев у каждого из них.

## Некоторые объяснения про регулярные выражения

* Что такое `re.compile`? <br><br>
Грубо говоря, `compile()` позволяет запомнить регулярное выражение и использовать его несколько раз. Суть в том, что перед тем как прогнать регулярку через строку, питон должен ее "скомпилировать" - превратить **строку** с регулярным выражением в специальный **объект**.<br>
Строчка `re.search(..., ...)` сначала компилирует регулярное выражение, а потом выполняет поиск. Если нужно поискать что-то один раз, то такая строчка очень удобна. А если нужно поискать что-то много раз, то получится что одно и то же выражение мы компилируем много раз. А хочется один раз скомпилировать и потом много раз пользоваться. Поэтому пишут так:

In [ ]:
text = 'тут текст, внутри которого мы что-то ищем'
regName = re.compile('тут регулярное выражение') # скомпилировали
toSearch = regName.search(text) # теперь можно искать в тексте
toFindAll = regName.findall(text)  # можно использовать скомпилированное выражение много раз
toSub = regName.sub('на.что.заменить', text) # и так тоже можно использовать

* Что делает `regName.sub(..., ...)`?<br><br>
Выражение  `regName.sub('на_что_заменить', text)` значит: возьми скомпилированное выражение из переменной `regName`, и замени все, что соответствует этому выражению в переменной `text`, на строку `'на_что_заменить'`. Если первый аргумент в этом случае - пустая строка, то все найденные регуляркой куски заменятся на пустую строку, короче говоря, удалятся.<br><br>

* Что такое `re.DOTALL`?<br><br>
Обычно точка в регулярном выражении означает любой символ КРОМЕ символа новой строки.  Чтобы изменить такое поведение, в компиляцию регулярки можно добавить параметры-флаги вот так: `flags = re.DOTALL`, и тогда точка будет ловить вообще любой символ, включая новую строку. Эти флаги слегка меняют поведение функции, вот и все.<br><br>

* Что такое `re.U`?<br><br>
U я написала зря, потому что я обычно пишу на втором питоне: во втором питоне по умолчанию выражения типа `\w`, `\W`, `\s` и подобные работают только на строках ASCII, и чтобы они работали на юникодных строках нужно написать re.U. В третьем питоне все строки и так юникодные, поэтому необходимости в таком флаге больше нет. Но если вдруг вам придется использовать регулярки из второго питона, запомните про такую штуку.
